# Fishing

## Step 0 - Importing libraries and reading the data

To start the project we need to import the most common libraries we will use for exploring the data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
df = pd.read_csv('../Resources/elektronisk-rapportering-ers-2018-fangstmelding-dca-simple.csv', delimiter=';')

## Step 1 - Data understanding

The initial step to machine learning is understanding the data we're working with. 

In [8]:
df.shape

(305434, 45)

In [9]:
df.head(5)

,Melding ID,Meldingstidspunkt,Meldingsdato,Meldingsklokkeslett,Starttidspunkt,Startdato,Startklokkeslett,Startposisjon bredde,Startposisjon lengde,Hovedområde start (kode),...,Art - FDIR,Art - gruppe (kode),Art - gruppe,Rundvekt,Lengdegruppe (kode),Lengdegruppe,Bruttotonnasje 1969,Bruttotonnasje annen,Bredde,Fartøylengde
0,1497177,01.01.2018,01.01.2018,00:00,31.12.2017,31.12.2017,00:00,"-60,35","-46,133",NaN,...,Antarktisk krill,506.0,Antarktisk krill,706714.0,5.0,28 m og over,9432.0,NaN,"19,87","133,88"
1,1497178,01.01.2018,01.01.2018,00:00,30.12.2017 23:21,30.12.2017,23:21,"74,885","16,048",20.0,...,Hyse,202.0,Hyse,9594.0,5.0,28 m og over,1476.0,NaN,"12,6","56,8"
2,1497178,01.01.2018,01.01.2018,00:00,30.12.2017 23:21,30.12.2017,23:21,"74,885","16,048",20.0,...,Torsk,201.0,Torsk,8510.0,5.0,28 m og over,1476.0,NaN,"12,6","56,8"
3,1497178,01.01.2018,01.01.2018,00:00,30.12.2017 23:21,30.12.2017,23:21,"74,885","16,048",20.0,...,Blåkveite,301.0,Blåkveite,196.0,5.0,28 m og over,1476.0,NaN,"12,6","56,8"
4,1497178,01.01.2018,01.01.2018,00:00,30.12.2017 23:21,30.12.2017,23:21,"74,885","16,048",20.0,...,Sei,203.0,Sei,134.0,5.0,28 m og over,1476.0,NaN,"12,6","56,8"


In [10]:
df.columns

Index(['Melding ID', 'Meldingstidspunkt', 'Meldingsdato',
       'Meldingsklokkeslett', 'Starttidspunkt', 'Startdato',
       'Startklokkeslett', 'Startposisjon bredde', 'Startposisjon lengde',
       'Hovedområde start (kode)', 'Hovedområde start',
       'Lokasjon start (kode)', 'Havdybde start', 'Stopptidspunkt',
       'Stoppdato', 'Stoppklokkeslett', 'Varighet', 'Fangstår',
       'Stopposisjon bredde', 'Stopposisjon lengde',
       'Hovedområde stopp (kode)', 'Hovedområde stopp',
       'Lokasjon stopp (kode)', 'Havdybde stopp', 'Trekkavstand',
       'Redskap FAO (kode)', 'Redskap FAO', 'Redskap FDIR (kode)',
       'Redskap FDIR', 'Hovedart FAO (kode)', 'Hovedart FAO',
       'Hovedart - FDIR (kode)', 'Art FAO (kode)', 'Art FAO',
       'Art - FDIR (kode)', 'Art - FDIR', 'Art - gruppe (kode)',
       'Art - gruppe', 'Rundvekt', 'Lengdegruppe (kode)', 'Lengdegruppe',
       'Bruttotonnasje 1969', 'Bruttotonnasje annen', 'Bredde',
       'Fartøylengde'],
      dtype='object')

In [11]:
df.dtypes

Melding ID                    int64
Meldingstidspunkt            object
Meldingsdato                 object
Meldingsklokkeslett          object
Starttidspunkt               object
Startdato                    object
Startklokkeslett             object
Startposisjon bredde         object
Startposisjon lengde         object
Hovedområde start (kode)    float64
Hovedområde start            object
Lokasjon start (kode)       float64
Havdybde start                int64
Stopptidspunkt               object
Stoppdato                    object
Stoppklokkeslett             object
Varighet                      int64
Fangstår                      int64
Stopposisjon bredde          object
Stopposisjon lengde          object
Hovedområde stopp (kode)    float64
Hovedområde stopp            object
Lokasjon stopp (kode)       float64
Havdybde stopp                int64
Trekkavstand                float64
Redskap FAO (kode)           object
Redskap FAO                  object
Redskap FDIR (kode)         

In [12]:
df.describe()

,Melding ID,Hovedområde start (kode),Lokasjon start (kode),Havdybde start,Varighet,Fangstår,Hovedområde stopp (kode),Lokasjon stopp (kode),Havdybde stopp,Trekkavstand,Redskap FDIR (kode),Hovedart - FDIR (kode),Art - FDIR (kode),Art - gruppe (kode),Rundvekt,Lengdegruppe (kode),Bruttotonnasje 1969,Bruttotonnasje annen
count,3.054340e+05,303433.000000,303433.000000,305434.000000,305434.000000,305434.000000,303472.000000,303472.000000,305434.000000,3.054100e+05,305246.000000,300456.000000,300452.000000,300452.000000,3.004560e+05,304750.000000,234005.000000,74774.000000
mean,1.658783e+06,14.463737,19.074712,-228.025292,537.095526,2017.999941,14.430415,18.883353,-229.084850,1.566397e+04,46.489746,1326.729934,1414.625914,259.746585,7.438208e+03,4.575032,1408.386975,186.172573
std,9.130738e+04,13.001244,18.469340,226.062493,2201.624688,0.007677,12.973150,18.361244,224.277365,9.033085e+04,13.534202,614.506560,633.188386,320.124913,4.281086e+04,0.692769,1148.384145,165.761157
min,1.497177e+06,0.000000,0.000000,-5388.000000,0.000000,2017.000000,0.000000,0.000000,-5388.000000,0.000000e+00,11.000000,412.000000,211.000000,101.000000,0.000000e+00,3.000000,104.000000,21.000000
25%,1.567228e+06,5.000000,7.000000,-273.000000,123.000000,2018.000000,5.000000,7.000000,-274.000000,2.533000e+03,32.000000,1022.000000,1022.000000,201.000000,6.400000e+01,4.000000,496.000000,87.000000
50%,1.674230e+06,8.000000,12.000000,-196.000000,296.000000,2018.000000,8.000000,12.000000,-198.000000,7.598000e+03,51.000000,1032.000000,1032.000000,203.000000,3.000000e+02,5.000000,1184.000000,149.000000
75%,1.735590e+06,20.000000,24.000000,-128.000000,494.000000,2018.000000,20.000000,24.000000,-127.000000,2.259900e+04,55.000000,1038.000000,2202.000000,302.000000,2.236000e+03,5.000000,2053.000000,236.000000
max,1.800291e+06,81.000000,87.000000,1220.000000,125534.000000,2018.000000,81.000000,87.000000,1616.000000,1.588863e+07,80.000000,6619.000000,6619.000000,9903.000000,1.100000e+06,5.000000,9432.000000,1147.000000


# Step 2 - Data preparation

In [13]:
df = df[[
    'Starttidspunkt','Startposisjon bredde', 'Startposisjon lengde', 'Hovedområde start',
    'Havdybde start', 'Stopptidspunkt', 'Varighet', 'Fangstår', 'Stopposisjon bredde', 
    'Stopposisjon lengde', 'Hovedområde stopp', 'Havdybde stopp', 'Trekkavstand', 'Redskap FDIR',
    'Art FAO', 'Art - gruppe', 'Rundvekt', 'Bruttotonnasje annen', 
    'Bruttotonnasje 1969', 'Bredde', 'Fartøylengde'
    ]].copy()

In [14]:
df['Starttidspunkt'] = pd.to_datetime(df['Starttidspunkt'], format='mixed')
df['Stopptidspunkt'] = pd.to_datetime(df['Stopptidspunkt'], format='mixed')

df['Fartøylengde'] = pd.to_numeric(df['Fartøylengde'].str.replace(',', '.'))
df['Bredde'] = pd.to_numeric(df['Bredde'].str.replace(',', '.'))

df['Startposisjon bredde'] = pd.to_numeric(df['Startposisjon bredde'].str.replace(',', '.'))
df['Startposisjon lengde'] = pd.to_numeric(df['Startposisjon lengde'].str.replace(',', '.'))

In [15]:
df.isna().sum()

Starttidspunkt               0
Startposisjon bredde         0
Startposisjon lengde         0
Hovedområde start         4124
Havdybde start               0
Stopptidspunkt               0
Varighet                     0
Fangstår                     0
Stopposisjon bredde          0
Stopposisjon lengde          0
Hovedområde stopp         4124
Havdybde stopp               0
Trekkavstand                24
Redskap FDIR               188
Art FAO                   4982
Art - gruppe              4982
Rundvekt                  4978
Bruttotonnasje annen    230660
Bruttotonnasje 1969      71429
Bredde                     684
Fartøylengde                 0
dtype: int64

In [20]:
df.loc[df.duplicated()]

,Starttidspunkt,Startposisjon bredde,Startposisjon lengde,Hovedområde start,Havdybde start,Stopptidspunkt,Varighet,Fangstår,Stopposisjon bredde,Stopposisjon lengde,...,Havdybde stopp,Trekkavstand,Redskap FDIR,Art FAO,Art - gruppe,Rundvekt,Bruttotonnasje annen,Bruttotonnasje 1969,Bredde,Fartøylengde
6338,2018-10-01 11:57:00,70.762,30.007,Øst-Finnmark,-77,2018-10-01 16:27:00,270,2018,"70,635","29,729",...,1,17494.0,Snurrevad,NaN,NaN,NaN,NaN,498.0,9.6,34.85
39762,2018-02-15 21:04:00,66.350,-16.850,Nordøst av Island,-220,2018-02-15 21:07:00,3,2018,"66,35","-16,85",...,-220,0.0,Snurpenot/ringnot,NaN,NaN,NaN,NaN,2832.0,15.2,74.20
40910,2018-02-16 04:06:00,57.997,5.575,Eigersundbanken,-242,2018-02-16 19:45:00,939,2018,"57,861","5,896",...,-249,24314.0,Reketrål,Dypvannsreke,Dypvannsreke,520.0,121.0,NaN,7.0,19.90
45207,2018-02-20 08:14:00,68.910,15.076,NaN,31,2018-02-20 10:02:00,108,2018,"68,91","15,076",...,31,0.0,Snurrevad,Torsk,Torsk,340.0,173.0,NaN,6.7,27.47
45208,2018-02-20 08:14:00,68.910,15.076,NaN,31,2018-02-20 10:02:00,108,2018,"68,91","15,076",...,31,0.0,Snurrevad,Sei,Sei,300.0,173.0,NaN,6.7,27.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292004,2018-05-12 03:00:00,71.530,28.301,Øst-Finnmark,-352,2018-05-12 19:19:00,979,2018,"71,53","28,305",...,-352,141.0,Andre liner,Annen skate og rokke,Skater og annen bruskfisk,400.0,48.0,NaN,5.5,15.01
292005,2018-05-12 03:00:00,71.530,28.301,Øst-Finnmark,-352,2018-05-12 19:19:00,979,2018,"71,53","28,305",...,-352,141.0,Andre liner,Flekksteinbit,Steinbiter,20.0,48.0,NaN,5.5,15.01
292006,2018-05-12 03:00:00,71.530,28.301,Øst-Finnmark,-352,2018-05-12 19:19:00,979,2018,"71,53","28,305",...,-352,141.0,Andre liner,Brosme,Annen torskefisk,20.0,48.0,NaN,5.5,15.01
292007,2018-05-12 03:00:00,71.530,28.301,Øst-Finnmark,-352,2018-05-12 19:19:00,979,2018,"71,53","28,305",...,-352,141.0,Andre liner,Blåkveite,Blåkveite,10.0,48.0,NaN,5.5,15.01


In [17]:
# Check for duplicate coaster name
df.loc[df.duplicated(subset=['Art FAO'])].head(5)

,Starttidspunkt,Startposisjon bredde,Startposisjon lengde,Hovedområde start,Havdybde start,Stopptidspunkt,Varighet,Fangstår,Stopposisjon bredde,Stopposisjon lengde,...,Havdybde stopp,Trekkavstand,Redskap FDIR,Art FAO,Art - gruppe,Rundvekt,Bruttotonnasje annen,Bruttotonnasje 1969,Bredde,Fartøylengde
5,2017-12-31 05:48:00,74.910,15.868,Bjørnøya,-403,2017-12-31 10:15:00,267,2017,"74,901","16,248",...,-277,11096.0,Bunntrål,Hyse,Hyse,9118.0,NaN,1476.0,12.6,56.8
6,2017-12-31 05:48:00,74.910,15.868,Bjørnøya,-403,2017-12-31 10:15:00,267,2017,"74,901","16,248",...,-277,11096.0,Bunntrål,Torsk,Torsk,6651.0,NaN,1476.0,12.6,56.8
7,2017-12-31 05:48:00,74.910,15.868,Bjørnøya,-403,2017-12-31 10:15:00,267,2017,"74,901","16,248",...,-277,11096.0,Bunntrål,Blåkveite,Blåkveite,130.0,NaN,1476.0,12.6,56.8
9,2017-12-31 05:48:00,74.910,15.868,Bjørnøya,-403,2017-12-31 10:15:00,267,2017,"74,901","16,248",...,-277,11096.0,Bunntrål,Sei,Sei,67.0,NaN,1476.0,12.6,56.8
11,2017-12-31 11:34:00,74.883,16.056,Bjørnøya,-346,2017-12-31 16:49:00,315,2017,"74,924","15,742",...,-496,10215.0,Bunntrål,Hyse,Hyse,12432.0,NaN,1476.0,12.6,56.8
